**Run the following two cells before you begin.**

In [ ]:
%autosave 100

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_auc_score
from sklearn.linear_model import LogisticRegression

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [4]:
# Import the data set
credit=pd.read_csv('cleaned_data.csv')
df=credit.copy()

In [5]:
# Define the sigmoid function
def sigmoid(a):
    x1=a[0]
    x2=a[1]
    b=a[2]
    z=(8.27451187e-11*x1)+(-6.80876727e-06*x2)+(-6.57647457e-11*b) # f(x)=wx+b w and b are intersepts and cofficent 

    sig= 1/(1+np.exp(z)) #sig =1/(1+e**(f(x)))
    return sig
#sigmoid is define as the racipocal the function of x is reciprocal of 1 plus exponent to the power minus x

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [6]:
X=df[['PAY_1','LIMIT_BAL']]
y=df['default payment next month']
# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [7]:
ml=LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [8]:
# Fit the logistic regression model on training data
ml.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [9]:
# Make predictions using `.predict()`
classes=ml.predict(X_test)

In [10]:
# Find class probabilities using `.predict_proba()`
probs=ml.predict_proba(X_test)[:,1]

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [11]:
# Add column of 1s to features
X_test['ones']=1

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
X_test

,PAY_1,LIMIT_BAL,ones
14306,2,160000,1
2978,1,50000,1
16641,-1,200000,1
18580,3,200000,1
131,1,50000,1
...,...,...,...
18160,0,140000,1
19779,-1,50000,1
6666,-1,50000,1
18990,1,230000,1


In [13]:
# Get coefficients and intercepts from trained model
print('cofficent ',ml.coef_)
print('\n')
print('intersept ',ml.intercept_)

cofficent  [[ 8.27451187e-11 -6.80876727e-06]]


intersept  [-6.57647457e-11]


In [14]:
X_test.head(1)

,PAY_1,LIMIT_BAL,ones
14306,2,160000,1


In [47]:
# Manually calculate predicted probabilities
manual_probs=1-X_test[['PAY_1','LIMIT_BAL','ones']].apply(sigmoid,axis=1) #using the def function created at top

In [48]:
manual_probs

14306    0.251731
2978     0.415703
16641    0.203955
18580    0.203955
131      0.415703
           ...   
18160    0.278236
19779    0.415703
6666     0.415703
18990    0.172785
23253    0.336066
Length: 5333, dtype: float64

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [49]:
# Manually calculate predicted classes
manual_classes=[]
for i in manual_probs:
    if i >= 0.5:
        manual_classes.append(1)
    elif i<0.5:
        manual_classes.append(0)

In [50]:
manual_probs

14306    0.251731
2978     0.415703
16641    0.203955
18580    0.203955
131      0.415703
           ...   
18160    0.278236
19779    0.415703
6666     0.415703
18990    0.172785
23253    0.336066
Length: 5333, dtype: float64

In [55]:
#manual_classes

In [52]:
# Compare to scikit-learn's predicted classes
np.unique(classes==manual_classes,return_counts=True)

(array([ True]), array([5333], dtype=int64))

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [53]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,probs)

0.627207450280691

In [54]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,manual_probs)

0.627207450280691